<a href="https://colab.research.google.com/github/jjpopski/JupyterNotebooks/blob/main/DopplerShift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Introduzione

Il moto relativo tra sorgente e osservatore causa una variazione della frequenza ricevuta dovuto all'effetto Doppler. Il moto dell'osservatore  rispetto alla sorgente ha vari contributi: 
- topocentrico (rotazione terrestre). 
- geocentrico (rivoluzione attorno al Sole)
- baricentrico (moto sistematico del baricentro del sistema solare versi Local Standard of Rest)

I cataloghi di radio sorgenti possono contenere l'informazione della velocità radiale. In genere, per le sorgenti radio galattiche  viene indicata la velocità radiale rispetto al LSR ($v_{LSR}$). 


# Correzione Doppler

Sia $v_{sun}$ la velocità del baricentro sel sistema oslare  verso l'apice, $\vec{u_{sa}}$ e $\vec{u_s}$ due vettori unitari rispettivamente verso la direzione dell'apice solare e della sorgente. La velocità radiale del sole nella direzione del LSR sarà pertanto data da: 

$$ v_{rs} = v_{sun}( \vec{u_{sa}} \cdot \vec{u_s} ) $$


Pertanto la velocità radiale $v_b$ baricentrica  è data da: 

$$ v_b = v_{LSR} − v_{rs} $$

Per calcolare il contributo geocentrico si procede analogamente noto il vettore di velocità della Terra $\vec{v_e}$

La velocità radiale della Terra in direzione della sorgente ($v_{re}$) è data dal prodotto scalare:
$$ v_{re} = ( \vec{v_{e}} \cdot \vec{u_s} ) $$

Il $\vec{v_{o}}$ è il vettore di velocita  dell'osservatore nel sistema di riferimento geocentrico equatoriale, pertanto il cui contributo nella direzione della sorgente è dato da :

$$ v_{ro} = ( \vec{v_{o}} \cdot \vec{u_s} ) $$ 

Considerando tutti i contributi alla velocita' radiale dovuti dal moto , la velocita' radiale topocentrica e':

$$ v_{t}= v_{LSR}- v_{rs} - v_{re}- v_{ro} = v_{b} - v_{re}- v_{ro}   $$





# Implementazione Python


L'implementazione richiede l'utilizzo del package Astropy versione 4 o superiore a cui si rimanda per la documentazione specifica, in particolare delle classi per la gestione del tempo e delle coordinate celesti.

La classe `SkyCoord` del package  astropy.coordinates permette di istanziare un oggetto rappresentativo
di un sorgente celeste, di coordinate RA e declinazione DEC.
```python
from astropy.coordinates import SkyCoord
source=SkyCoord (ra=RA,  dec= DEC, frame='icrs')


```

Il metodo `radial_velocity_correction` permette di ottenere $v_{ob}$ la velocita' radiale nella direzione di una della sorgente rispetto al baricentro del sistema solare, a un tempo `t` in un punto della superficie terrestre `loc`


```python

velocity= source.radial_velocity_correction(obstime=time, location=loc)


```

Nelle definizioni di cui sopra, $v_{ob}$ rappresenta la somma del contributo geocentrico e quello baricentrico

$$v_{ob}= v_{re}+ v_{ro}$$


$$ v_{t}= v_{LSR}- v_{rs} - v_{ob} $$

Per definzione, il Local Standard of Rest ha velocita' $v_{LSR}=0$, pertanto la velocita' dell'
osservatore rispetto a LSR e= data da

$$ v_{t}=- v_{rs} - v_{ob} $$

La correzione da impostare sull'oscillatore locale :

$$ \Delta f_{OL}= - f_{obs}\frac{v_t}{c}$$

dove $f_{obs}$ e' la frequenza cielo e *c* e' la velocita' della luce.

## Bibliografia


- A. Díaz-Pulido, P. de Vicente Spectral data doppler corrections for the 40 m radiotelescope, Informe Técnico IT-CDT/OAN 2014-10

- M.A. Gordon 1975, Methods of Experimental Physics: Volume 12: Astrophysics, Part C: Radio Observations - Section 6.1.5.
- https://docs.astropy.org/en/stable/index.html




In [1]:
from astropy import units as u
from astropy.coordinates import SkyCoord,EarthLocation,LSRK,ICRS
from astropy.time import Time
import numpy as np
import astropy.constants
#%xmode Verbose


In [2]:
time=Time('2021-08-20T15:38:45.672')  

Il Local Standard of Rest (LSR) è un sistema di riferimento che tiene conto dei moti sistematici delle stelle nei dintorni del Sole. Si defilnisce LSR dinamico o LSRK il sistema di riferimento tale per cui il Sole si muove ad una velcotà di 20 Km/s verso l'apice solare che si trova alle  coordinate  R.A.=$18^h$ e Dec=30$^\circ$ nel sistema di riferimento equatoriale all'epoca J1900. Nell'International Celestial Reference System (ICRS), l'apice solare ha coordinate R.A. = $"18^h03^m50^s.29$, Dec = $+30^\circ00^{'}16^{"}.8$, all'epoca J2000.


In [3]:
psun = SkyCoord(ra="18:03:50.29", dec="+30:00:16.8", frame="icrs", unit=(u.hourangle, u.deg))

vsun = -20.0 * u.km / u.s


 

In [4]:
location = EarthLocation.of_site('SRT')  
source=SkyCoord.from_name('W3(OH)')
print(location.geodetic)

w3oh=SkyCoord.from_name('W3(OH)')
aries_point=SkyCoord(ra=0*u.degree,dec=0*u.degree,frame='icrs')



GeodeticLocation(lon=<Longitude 9.24516 deg>, lat=<Latitude 39.49304 deg>, height=<Quantity 650. m>)


In [5]:
def vlsr(source,time,location):
      vsrc = source.radial_velocity_correction(kind='barycentric',obstime=time, location=location)

      # Projection of solar velocity towards the source
      vsun_proj = source.cartesian.dot(psun.cartesian) * vsun
      return vsrc-vsun_proj

def deltafreq(fobs,vt):
    return -fobs*vt/astropy.constants.c

In [6]:
print(vlsr(w3oh,time,location).to(u.km/u.s))
print(vlsr(aries_point,time,location).to(u.km/u.s))



25.127571305055895 km / s
15.97246901294389 km / s


In [7]:
fobs=22235.079850*u.MHz #water maser frequency. 
vt_w3oh=vlsr(w3oh,time,location)
print(deltafreq(fobs,vt_w3oh))


-1.8636678124987605 MHz
